In [20]:
import numpy as np
import pandas as pd
import os
# Lista de posibles rutas
addresses = [
    'C:/Users/RONALD Q/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/DGA.xlsx',
    'C:/Users/roquispec/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/DGA.xlsx',
    'C:/Users/mticllacu/OneDrive - LUZ DEL SUR S.A.A/Archivos de Ronald Quispe Ocaña - ProyectoRyD_V2/Basededatos/DGA.xlsx'
]

df = None
for path in addresses:
    if os.path.exists(path):   # verifica si existe
        df = pd.read_excel(path,header=1)
        print(f"✅ Archivo cargado desde: {path}")
        break

if df is None:
    raise FileNotFoundError("❌ No se encontró el archivo en ninguna de las rutas especificadas.")
df["SERIE"] = df["SERIE"].astype(str)
df['SERIE'] = df['SERIE'].astype(str).str.replace(" ", "")
df.head()

✅ Archivo cargado desde: C:/Users/roquispec/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/DGA.xlsx


,Unnamed: 0,SERIE,FECHA DE MUESTRA,H2,CH4,C2H2,C2H4,C2H6,CO,CO2,O2,N2
0,NaN,D518293,2025-02-07 00:00:00,4.0,11.8,1.0,1.0,2.6,1064.6,3681.0,3372.2,28075.0
1,NaN,D518293,2023-08-29 00:00:00,3.0,9.0,1.0,1.0,2.0,788.0,3213.0,10157.0,60776.0
2,NaN,D518293,2022-06-14 00:00:00,5.0,9.0,1.0,1.0,2.0,758.0,3021.0,3106.0,20091.0
3,NaN,D518293,2022-04-28 00:00:00,1.0,8.0,1.0,1.0,2.0,745.0,2942.0,3837.0,34777.0
4,NaN,D518293,2021-08-15 00:00:00,7.0,7.0,1.0,1.0,1.0,547.0,2375.0,11922.0,50445.0


In [21]:
# ---------------------------
# LIMPIEZA DE DATOS
# ---------------------------
df = df.drop(columns=['Unnamed: 0'])
# df["TENSION"] = df["TENSION"].str.split("/").str[0]
df = df.iloc[:, :10]
if 'FECHA DE MUESTRA' in df.columns:
    df = df.rename(columns={'FECHA DE MUESTRA': 'FECHA'})
elif 'FECHA DE\nMUESTRA' in df.columns:
    df = df.rename(columns={'FECHA DE\nMUESTRA': 'FECHA'})
df['FECHA'] = pd.to_datetime(df['FECHA'], errors='coerce')
df = df.dropna(subset=['FECHA'])
df["SERIE"] = df["SERIE"].astype(str)
# Guardar tabla original (solo fechas de medición)
# df_full = df.drop(columns=["TENSION"]).copy()
df_full = df.copy()
# ---------------------------
# LÍMITES Y GASES
# ---------------------------
limite = {
    'H2': {'concentracion': 150, 'tasa': 132},
    'CH4': {'concentracion': 130, 'tasa': 120},
    "C2H2": {"concentracion": 20, "tasa": 4},
    "C2H4": {"concentracion": 280, "tasa": 146},
    "C2H6": {"concentracion": 90, "tasa": 90},
    "CO": {"concentracion": 600, "tasa": 1060},
    "CO2": {"concentracion": 14000, "tasa": 10000}
}
gases2 = list(limite.keys())

# ---------------------------
# CONCENTRACIÓN
# ---------------------------
for gas in gases2:
    df[f'concentracion_{gas}'] = df.apply(lambda g: int(g[gas] > limite[gas]['concentracion']), axis=1)

# ---------------------------
# TASA DE INCREMENTO ANUAL
# ---------------------------
for gas in gases2:
    df[f"tasa_{gas}"] = df.groupby("SERIE").apply(
        lambda g: (g[gas].diff(-1) / ((g["FECHA"] - g["FECHA"].shift(-1)).dt.days) * 365)
    ).reset_index(level=0, drop=True)

# ---------------------------
# FLAGS Y PUNTAJE
# ---------------------------
for gas, valores in limite.items():
    df[f"flag_{gas}"] = (df[f"tasa_{gas}"] > valores['tasa']).astype(int)

def puntaje_gas(row, gas):
    conc = row[f'concentracion_{gas}']
    tasa = row[f'flag_{gas}']
    if conc > 0 and tasa > 0:
        return 5
    elif conc > 0 and tasa <= 0:
        return 4
    elif conc <= 0 and tasa > 0:
        return 3
    else:
        return 1

for gas in gases2:
    df[f'puntaje_{gas}'] = df.apply(lambda row: puntaje_gas(row, gas), axis=1)

# ---------------------------
# CÁLCULO DGA
# ---------------------------
weight = {"C2H2":5,"H2":2,"CH4":3,"C2H4":4,"C2H6":3,"CO":2,"CO2":1}
total_weight = sum(weight.values())
df['DGA'] = df.apply(lambda row: sum(row[f'puntaje_{gas}']*weight[gas] for gas in gases2)/total_weight, axis=1)

df_DGA = df[['SERIE','FECHA','DGA']]

# ---------------------------
# EXTENSIÓN DEL CALENDARIO DESDE 2025
# ---------------------------
inicio = "2015-01-01"
desde_2025 = f"{pd.Timestamp.today().year}-01-01"
fecha_inicio = pd.Timestamp(inicio)  # en el 2026 cambiar ---****
fecha_fin = pd.Timestamp.today().normalize()
fechas = pd.date_range(fecha_inicio, fecha_fin, freq="D")

# Calendario
todas_series = df['SERIE'].dropna().unique()
calendario = pd.MultiIndex.from_product([todas_series, fechas], names=["SERIE","FECHA"])
df_calendario = pd.DataFrame(index=calendario).reset_index()

# ---------- Tabla extendida de DGA ----------
ultimos_2024 = df_DGA[df_DGA['FECHA'] < fecha_inicio].sort_values('FECHA').groupby('SERIE').tail(1)
ultimos_2024['FECHA'] = fecha_inicio
base_ext = pd.concat([df_DGA, ultimos_2024], ignore_index=True)

df_extendida = pd.merge(df_calendario, base_ext, on=["SERIE","FECHA"], how="left")
df_extendida = df_extendida.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)

# ---------- Tabla extendida de detalles ----------
ultimos_2024_det = df_full[df_full['FECHA'] < fecha_inicio].sort_values('FECHA').groupby('SERIE').tail(1)
ultimos_2024_det['FECHA'] = fecha_inicio
base_ext_det = pd.concat([df_full, ultimos_2024_det], ignore_index=True)

df_extendida_detalles = pd.merge(df_calendario, base_ext_det, on=["SERIE","FECHA"], how="left")
df_extendida_detalles = df_extendida_detalles.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)

# ---------------------------
# DETALLES + DGA
# ---------------------------
df_detalles = pd.merge(df_full, df_DGA, on=["SERIE","FECHA"], how="left")
df_detalles_ext = pd.merge(df_extendida_detalles, df_extendida, on=["SERIE","FECHA"], how="left")

# Reordenar columnas: poner DGA después de FECHA
def reordenar(df_in):
    cols = list(df_in.columns)
    if "DGA" in cols:
        cols.remove("DGA")
        idx = cols.index("FECHA") + 1
        cols = cols[:idx] + ["DGA"] + cols[idx:]
    return df_in[cols]

df_detalles = reordenar(df_detalles)
df_detalles_ext = reordenar(df_detalles_ext)

# ---------------------------
# FUNCIONES PARA LLAMAR
# ---------------------------
def get_df_DGA():
    return df_DGA

def get_df_extendida_DGA():
    return df_extendida

def get_df_detalles_DGA():
    return df_detalles

def get_df_detalles_ext_DGA():
    return df_detalles_ext

C:\Users\roquispec\AppData\Local\Temp\ipykernel_20168\3387753875.py:41: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[f"tasa_{gas}"] = df.groupby("SERIE").apply(
C:\Users\roquispec\AppData\Local\Temp\ipykernel_20168\3387753875.py:41: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[f"tasa_{gas}"] = df.groupby("SERIE").apply(
C:\Users\roquispec\AppData\Local\Temp\ipykernel_20168\3387753875.py:41: FutureWarning: Dat

In [22]:
df_detalles.head(20)
df = df_detalles.copy()
df.head(20)

,SERIE,FECHA,DGA,H2,CH4,C2H2,C2H4,C2H6,CO,CO2,O2
0,D518293,2025-02-07,1.3,4.0,11.8,1.0,1.0,2.6,1064.6,3681.0,3372.2
1,D518293,2023-08-29,1.3,3.0,9.0,1.0,1.0,2.0,788.0,3213.0,10157.0
2,D518293,2022-06-14,1.3,5.0,9.0,1.0,1.0,2.0,758.0,3021.0,3106.0
3,D518293,2022-04-28,1.3,1.0,8.0,1.0,1.0,2.0,745.0,2942.0,3837.0
4,D518293,2021-08-15,1.0,7.0,7.0,1.0,1.0,1.0,547.0,2375.0,11922.0
5,D518293,2020-06-23,1.0,10.0,5.0,1.0,1.0,1.0,424.0,1467.0,2609.0
6,D518293,2019-04-25,1.0,1.0,4.0,1.0,1.0,1.0,349.0,1281.0,18414.0
7,D518293,2018-08-03,1.0,1.0,1.0,1.0,1.0,1.0,135.0,422.0,1676.0
8,D518293,2017-11-15,1.0,1.0,1.0,1.0,1.0,1.0,44.0,213.0,2396.0
9,D518293,2017-10-25,1.0,1.0,1.0,1.0,1.0,1.0,18.0,98.0,3007.0


In [23]:
# =============================================================================
# 1. DEFINIR FEATURES DIRECTAMENTE
# =============================================================================

# Usar SOLO los gases que ya tienes en tu tabla
features_gases = ['H2', 'CO2', 'C2H2', 'CH4', 'C2H4', 'C2H6', 'CO']

# Verificar qué gases realmente existen en tu tabla
features_existentes = [gas for gas in features_gases if gas in df.columns]
print(f"✅ Gases disponibles: {features_existentes}")

# Features para el modelo: Los gases + el índice DGA que ya calculaste
features = features_existentes + ['DGA']

print(f"🎯 Features finales: {features}")

✅ Gases disponibles: ['H2', 'CO2', 'C2H2', 'CH4', 'C2H4', 'C2H6', 'CO']
🎯 Features finales: ['H2', 'CO2', 'C2H2', 'CH4', 'C2H4', 'C2H6', 'CO', 'DGA']


In [24]:
# =============================================================================
# 2. CREAR EL TARGET (ALERTA FUTURA)
# =============================================================================

print("🎯 Creando variable objetivo...")
df['target_alerta'] = 0

for serie in df['SERIE'].unique():
    serie_data = df[df['SERIE'] == serie].sort_values('FECHA') # Ordena las fechas
    
    for i in range(len(serie_data)-1):    # bucle recorre filas de una serie
        future_dga = serie_data.iloc[i+1]['DGA']  # DGA de la próxima medición
        if future_dga >= 1.3:  # Si en el futuro supera 1.3
            current_idx = serie_data.index[i]
            df.loc[current_idx, 'target_alerta'] = 1

print(f"✅ Target creado - Alertas: {df['target_alerta'].sum()}")
df.head(20)

🎯 Creando variable objetivo...
✅ Target creado - Alertas: 1144


,SERIE,FECHA,DGA,H2,CH4,C2H2,C2H4,C2H6,CO,CO2,O2,target_alerta
0,D518293,2025-02-07,1.3,4.0,11.8,1.0,1.0,2.6,1064.6,3681.0,3372.2,0
1,D518293,2023-08-29,1.3,3.0,9.0,1.0,1.0,2.0,788.0,3213.0,10157.0,1
2,D518293,2022-06-14,1.3,5.0,9.0,1.0,1.0,2.0,758.0,3021.0,3106.0,1
3,D518293,2022-04-28,1.3,1.0,8.0,1.0,1.0,2.0,745.0,2942.0,3837.0,1
4,D518293,2021-08-15,1.0,7.0,7.0,1.0,1.0,1.0,547.0,2375.0,11922.0,1
5,D518293,2020-06-23,1.0,10.0,5.0,1.0,1.0,1.0,424.0,1467.0,2609.0,0
6,D518293,2019-04-25,1.0,1.0,4.0,1.0,1.0,1.0,349.0,1281.0,18414.0,0
7,D518293,2018-08-03,1.0,1.0,1.0,1.0,1.0,1.0,135.0,422.0,1676.0,0
8,D518293,2017-11-15,1.0,1.0,1.0,1.0,1.0,1.0,44.0,213.0,2396.0,0
9,D518293,2017-10-25,1.0,1.0,1.0,1.0,1.0,1.0,18.0,98.0,3007.0,0


In [25]:
# =============================================================================
# 3. ENTRENAR MODELO DIRECTAMENTE
# =============================================================================

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Limpiar datos infinitos/NaN
df_clean = df.replace([np.inf, -np.inf], np.nan)
df_clean = df_clean.dropna(subset=features + ['target_alerta'])

X = df_clean[features]
y = df_clean['target_alerta']

print(f"📊 Datos para entrenamiento: {len(X)} registros")

if y.sum() > 5:  # Mínimo 5 alertas
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Evaluar
    y_pred = model.predict(X_test)
    print(f"🎯 Precisión: {accuracy_score(y_test, y_pred):.2f}")
    print("\n📈 Reporte:")
    print(classification_report(y_test, y_pred))
    
    # Features importantes
    importancias = pd.DataFrame({
        'feature': features,
        'importancia': model.feature_importances_
    }).sort_values('importancia', ascending=False)
    
    print("🏆 Features más importantes:")
    print(importancias)
    
else:
    print("❌ No hay suficientes alertas para entrenar")

📊 Datos para entrenamiento: 2682 registros
🎯 Precisión: 0.80

📈 Reporte:
              precision    recall  f1-score   support

           0       0.82      0.84      0.83       309
           1       0.78      0.75      0.77       228

    accuracy                           0.80       537
   macro avg       0.80      0.80      0.80       537
weighted avg       0.80      0.80      0.80       537

🏆 Features más importantes:
  feature  importancia
6      CO     0.185917
3     CH4     0.177697
1     CO2     0.167827
7     DGA     0.157581
5    C2H6     0.106689
0      H2     0.096159
4    C2H4     0.078233
2    C2H2     0.029897


In [26]:
df.head()

,SERIE,FECHA,DGA,H2,CH4,C2H2,C2H4,C2H6,CO,CO2,O2,target_alerta
0,D518293,2025-02-07,1.3,4.0,11.8,1.0,1.0,2.6,1064.6,3681.0,3372.2,0
1,D518293,2023-08-29,1.3,3.0,9.0,1.0,1.0,2.0,788.0,3213.0,10157.0,1
2,D518293,2022-06-14,1.3,5.0,9.0,1.0,1.0,2.0,758.0,3021.0,3106.0,1
3,D518293,2022-04-28,1.3,1.0,8.0,1.0,1.0,2.0,745.0,2942.0,3837.0,1
4,D518293,2021-08-15,1.0,7.0,7.0,1.0,1.0,1.0,547.0,2375.0,11922.0,1
